# keyness

> Functionality for keyness analysis.
- toc: false
- page-layout: full

In [ ]:
#| default_exp keyness

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
import numpy as np
import time
import polars as pl
from fastcore.basics import patch
from scipy.stats import chi2
import re

In [ ]:
#| export
from conc.corpus import Corpus
from conc.listcorpus import ListCorpus
from conc.result import Result
from conc.frequency import Frequency
from conc.core import logger, PAGE_SIZE

In [ ]:
#| hide
from conc.core import set_logger_state

In [ ]:
#| hide
polars_conf = pl.Config.set_tbl_cols(20)

In [ ]:
#| hide
import os

In [ ]:
#| hide
source_path = f'{os.environ.get("HOME")}/data/'
save_path = f'{os.environ.get("HOME")}/data/conc-test-corpora/'

path_to_toy_corpus = f'{save_path}toy.corpus'
path_to_brown_corpus = f'{save_path}brown.corpus'
path_to_brown_listcorpus = f'{save_path}brown.listcorpus'
path_to_reuters_corpus = f'{save_path}reuters.corpus'
path_to_gutenberg_corpus = f'{save_path}gutenberg.corpus'
path_to_gardenparty_corpus = f'{save_path}garden-party.corpus'
path_to_gardenparty_listcorpus = f'{save_path}garden-party.listcorpus'

## About Conc's Keyness functionality

Conc implements Log Ratio, which is a keyness measure introduced by Andrew Hardie in this ["informal introduction"](https://cass.lancs.ac.uk/log-ratio-an-informal-introduction/). In that piece Hardie also discusses Relative Risk, which is also implemented in Conc. The Log Likelihood Ratio implementation is based on discussion in [Paul Rayson's Log-likelihood and effect size calculator](https://ucrel.lancs.ac.uk/llwizard.html). Rayson discusses a variety of keyness measures and practical issues in implementing these. Conc follows the approach mentioned by Rayson of applying an observed frequency of 0.5 for words that do not appear in the target or reference corpus to avoid division by zero issues. Conc also follows handling of zero values in calculating log likelihood (see [Note 2](https://ucrel.lancs.ac.uk/llwizard.html)).

## Using the Keyness class

There are examples below showing how to use the Keyness class directly to output keyword tables. The recommended way to use this functionality is through the [Conc](https://geoffford.nz/conc/api/conc.html) class. This provides an interface to create frequency lists, concordances, collocation tables, keyword tables and more. 

## Keyness class API reference

In [ ]:
#| export
class Keyness:
	""" Class for keyness analysis reporting. """
	def __init__(self,
			  corpus:Corpus | ListCorpus, # Corpus instance
			  reference_corpus:Corpus | ListCorpus # Corpus for comparison
			  ): 
		self.corpus = corpus
		self.reference_corpus = reference_corpus	


In [ ]:
#| export
@patch
def keywords(self: Keyness,
				effect_size_measure:str = 'log_ratio', # effect size measure to use, currently only 'log_ratio' is supported
				statistical_significance_measure:str = 'log_likelihood', # statistical significance measure to use, currently only 'log_likelihood' is supported
				order:str|None = None, # default of None orders by statistical significance measure, results can also be ordered by: frequency, frequency_reference, document_frequency, document_frequency_reference, log_likelihood
				order_descending:bool = True, # order is descending or ascending
				statistical_significance_cut: float|None = None, # statistical significance p-value to filter results, e.g. 0.05 or 0.01 or 0.001 - ignored if None or 0
				apply_bonferroni:bool = False, # apply Bonferroni correction to the statistical significance cut-off
				min_document_frequency: int = 0, # minimum document frequency in target for token to be included in the report
				min_document_frequency_reference: int = 0, # minimum document frequency in reference for token to be included in the report
				min_frequency: int = 0, # minimum frequency in target for token to be included in the report
				min_frequency_reference: int = 0, # minimum document frequency in reference for token to be included in the report
				case_sensitive:bool=False, # frequencies for tokens with or without case preserved 
				normalize_by:int=10000, # normalize frequencies by a number (e.g. 10000)
				page_size:int=PAGE_SIZE, # number of rows to return, if 0 returns all
				page_current:int=1, # current page, ignored if page_size is 0
				show_document_frequency:bool=False, # show document frequency in output
				exclude_tokens:list[str]=[], # exclude specific tokens from report results
				exclude_tokens_text:str = '', # text to explain which tokens have been excluded, will be added to the report notes
				restrict_tokens:list[str]=[], # restrict report to return results for a list of specific tokens (see note below)
				restrict_tokens_text:str = '', # text to explain which tokens are included, will be added to the report notes
				exclude_punctuation:bool=True, # exclude punctuation tokens
				handle_common_typographic_differences:bool=True, # whether to detect and normalize common differences in word tokens due to typographic differences (i.e. currently focused on apostrophes in common English contractions), ignored when exclude_punctuation is False
				exclude_negative_keywords:bool=True # whether to exclude negative keywords from the report
				) -> Result: # return a Result object with the frequency table
	""" Get keywords for the corpus. """

	if type(normalize_by) != int:
		raise ValueError('normalize_by must be an integer, e.g. 1000000 or 10000')

	if effect_size_measure not in ['log_ratio']:
		raise ValueError('Currently only log_ratio is supported as an effect size measures.')
	
	if statistical_significance_measure not in ['log_likelihood']:
		raise ValueError('Currently only log_likelihood is supported as a statistical significance measure.')

	if order not in [None, effect_size_measure, 'frequency', 'frequency_reference', 'document_frequency', 'document_frequency_reference', statistical_significance_measure]:
		raise ValueError(f'The order parameter must None (default) or one of: {effect_size_measure}, frequency, frequency_reference, document_frequency, document_frequency_reference, {statistical_significance_measure}.')
	
	if not show_document_frequency and order in ['document_frequency', 'document_frequency_reference']:
		raise ValueError('The show_document_frequency parameter bust be set True if you want to order by document_frequency or document_frequency_reference.')

	if type(normalize_by) != int:
		raise ValueError('normalize_by must be an integer, e.g. 1000000 or 10000')

	start_time = time.time()

	if page_size == 0:
		page_current = 1 # if returning all, then only interested in first page

	debug = False

	freq_target = Frequency(self.corpus)
	freq_reference = Frequency(self.reference_corpus)

	columns = ['rank', 'token', 'frequency', 'frequency_reference', 'document_frequency', 'document_frequency_reference', 'normalized_frequency', 'normalized_frequency_reference']

	target_count_tokens, tokens_descriptor, total_descriptor = self.corpus.get_token_count_text(exclude_punctuation=exclude_punctuation)
	reference_count_tokens, _, _ = self.reference_corpus.get_token_count_text(exclude_punctuation=exclude_punctuation)

	formatted_data = []
	formatted_data.append(f'Report based on {tokens_descriptor}')

	target_min_freq = (0.5 * normalize_by) / target_count_tokens
	reference_min_freq = (0.5 * normalize_by) / reference_count_tokens
	
	target_df = freq_target.frequencies(case_sensitive=case_sensitive,
										normalize_by=normalize_by,
										page_size=0,
										page_current=1,
										show_token_id=False,
										show_document_frequency=True, # applying to final report not import
										exclude_tokens=exclude_tokens,
										exclude_tokens_text=exclude_tokens_text,
										restrict_tokens=restrict_tokens,
										restrict_tokens_text=restrict_tokens_text,
										exclude_punctuation=exclude_punctuation).to_frame()

	reference_df = freq_reference.frequencies(case_sensitive=case_sensitive,
										normalize_by=normalize_by,
										page_size=0,
										page_current=1,
										show_token_id=False,
										show_document_frequency=True, # applying to final report not import
										exclude_tokens=exclude_tokens,
										exclude_tokens_text=exclude_tokens_text,
										restrict_tokens=restrict_tokens,
										restrict_tokens_text=restrict_tokens_text,
										exclude_punctuation=exclude_punctuation).to_frame()

	if handle_common_typographic_differences is True:
		if exclude_punctuation == False:
			logger.warning(f'handle_common_typographic_differences is ignored when exclude_punctuation is False')
		else:
			valid_apostrophe_characters = ["’", "'"] # "‘", "’", "‘", "`"
			target_top_tokens_with_apostrophe = target_df.filter(pl.col('token').str.contains_any(valid_apostrophe_characters)).filter(pl.col('rank') < 1000).select(pl.col('token')).rows()
			target_top_tokens_with_apostrophe = [token[0] for token in target_top_tokens_with_apostrophe]
			apostrophe_characters = re.sub(r'[A-Za-z]', '', ''.join(target_top_tokens_with_apostrophe))
			apostrophe_characters = ''.join(set(apostrophe_characters))
			if len(apostrophe_characters) > 1:
				logger.warning(f'handle_common_typographic_differences, Target corpus uses multiple apostrophe characters, no further action taken')
			elif len(apostrophe_characters) == 0:
				logger.info(f'handle_common_typographic_differences, No frequent tokens with apostrophe characters, no further action taken')
			else:
				logger.info(f'handle_common_typographic_differences, Target corpus using apostrophe character: {apostrophe_characters}')
				reference_top_tokens_with_apostrophe = reference_df.filter(pl.col('token').str.contains_any(valid_apostrophe_characters)).filter(pl.col('rank') < 1000).select(pl.col('token')).rows()
				reference_top_tokens_with_apostrophe = ''.join([token[0] for token in reference_top_tokens_with_apostrophe])
				reference_apostrophe_characters = re.sub(r'[A-Za-z]', '', reference_top_tokens_with_apostrophe)
				reference_apostrophe_characters = ''.join(set(reference_apostrophe_characters))
				if len(reference_apostrophe_characters) != 1:
					logger.info(f'handle_common_typographic_differences, Reference corpus has multiple (or no) apostrophe characters in frequent word tokens, no further action taken, consider building corpus with standardize_word_token_punctuation_characters=True')
				elif reference_apostrophe_characters == apostrophe_characters:
					logger.info(f'handle_common_typographic_differences, Reference corpus has the same apostrophe character as target corpus ({apostrophe_characters}), no further action taken')
				else:
					replacements = {}
					for token in target_top_tokens_with_apostrophe:
						for valid_apostrophe_character in valid_apostrophe_characters:
							if valid_apostrophe_character != apostrophe_characters:
								replacements[token.replace(apostrophe_characters, valid_apostrophe_character)] = token
					logger.info(f'handle_common_typographic_differences, Replacing apostrophe characters in reference corpus tokens: {replacements}')
					reference_df = reference_df.with_columns(
						pl.col('token').replace(
							pl.Series(replacements.keys()), 
							pl.Series(replacements.values())).alias('token')
					)
					formatted_data.append(f'Frequent tokens with apostrophes have been normalized in reference corpus to match target usage')

	keyness_df = target_df.join(reference_df, on='token', how='left', suffix = '_reference', maintain_order='left').drop('rank', 'rank_reference')

	if exclude_tokens:
		excluded_tokens_count = keyness_df.filter(pl.col('token').is_in(exclude_tokens)).select(pl.len()).item()
		#keyness_df = keyness_df.filter(~pl.col('token').is_in(exclude_tokens))
		if exclude_tokens_text == '':
			formatted_data.append(f'Tokens excluded from report: {excluded_tokens_count}')
		else:
			formatted_data.append(f'{exclude_tokens_text}')

	if restrict_tokens:
		#keyness_df = keyness_df.filter(pl.col('token').is_in(restrict_tokens))
		if restrict_tokens_text == '':
			formatted_data.append(f'')
		else:
			formatted_data.append(f'{restrict_tokens_text}')

	keyness_df = keyness_df.with_columns(pl.col('frequency_reference')).fill_null(0)
	keyness_df = keyness_df.with_columns(pl.col('document_frequency_reference')).fill_null(0)

	if effect_size_measure in ['log_ratio']:
		columns.extend(['relative_risk', 'log_ratio'])
		
		keyness_df = keyness_df.with_columns(pl.lit(target_count_tokens).alias('token_count')).with_columns(pl.lit(reference_count_tokens).alias('token_count_reference'))
	
		# copying for calculation - will retain original values for display
		keyness_df = keyness_df.with_columns(pl.col('normalized_frequency').alias('calc_normalized_frequency'))
		keyness_df = keyness_df.with_columns(pl.col('normalized_frequency_reference').alias('calc_normalized_frequency_reference'))

		keyness_df = keyness_df.with_columns(pl.col('calc_normalized_frequency').replace(0, target_min_freq))
		keyness_df = keyness_df.with_columns(pl.col('calc_normalized_frequency_reference').replace(0, reference_min_freq))

		keyness_df = keyness_df.with_columns((pl.col('calc_normalized_frequency')/pl.col('calc_normalized_frequency_reference')).alias('relative_risk'))
		keyness_df = keyness_df.with_columns((pl.col('calc_normalized_frequency').log(2) - pl.col('calc_normalized_frequency_reference').log(2)).alias('log_ratio'))

	if statistical_significance_measure in ['log_likelihood']:
		columns.extend(['log_likelihood'])

		# calculating using approach here: https://ucrel.lancs.ac.uk/llwizard.html
		# a = frequency in target , b = frequency in reference 
		# c = total tokens in target , d = total tokens in reference 
		# E1 = c*(a+b) / (c+d) 
		# E2 = d*(a+b) / (c+d)
		# G2 = 2*((a*ln (a/E1)) + (b*ln (b/E2))) 
		
		# E1 and E2
		keyness_df = keyness_df.with_columns(
			((pl.col('token_count') * (pl.col('frequency') + pl.col('frequency_reference'))) / (pl.col('token_count') + pl.col('token_count_reference'))).alias('expected_frequency'),
			((pl.col('token_count_reference') * (pl.col('frequency') + pl.col('frequency_reference'))) / (pl.col('token_count') + pl.col('token_count_reference'))).alias('expected_frequency_reference'), # 0 if no reference frequency
		)

		# components of G2 as term1 and term 2 - (a*ln (a/E1)) (b*ln (b/E2))
		keyness_df = keyness_df.with_columns([
			pl.when(pl.col('frequency') > 0)
			.then(pl.col('frequency') * (pl.col('frequency') / pl.col('expected_frequency')).log())
			.otherwise(0)
			.alias('term1'),
			pl.when(pl.col('frequency_reference') > 0)
			.then(pl.col('frequency_reference') * (pl.col('frequency_reference') / pl.col('expected_frequency_reference')).log())
			.otherwise(0)
			.alias('term2') # 0 if no reference frequency
		])

		#G2
		keyness_df = keyness_df.with_columns(
			(2 * (pl.col('term1') + pl.col('term2'))).alias('log_likelihood')
		)

		# not needed - as use cutoff instead
		# combined_frequency_table = combined_frequency_table.collect()
		# combined_frequency_table = combined_frequency_table.with_columns(
		#     pl.Series("p_value", chi2.sf(combined_frequency_table["log_likelihood"].to_numpy(), 1)).alias("p_value")
		# )

	# filtering - must be done before bonferroni or similar correction ...
	filtering_descriptors = []
	if min_frequency > 0:
		keyness_df = keyness_df.filter(pl.col('frequency') >= min_frequency)
		filtering_descriptors.append(f'minimum frequency in target corpus ({min_frequency:,.0f})')
	if min_frequency_reference > 0:
		keyness_df = keyness_df.filter(pl.col('frequency_reference') >= min_frequency_reference)
		filtering_descriptors.append(f'minimum frequency in reference corpus ({min_frequency_reference:,.0f})')
	if min_document_frequency > 0:
		keyness_df = keyness_df.filter(pl.col('document_frequency') >= min_document_frequency)
		filtering_descriptors.append(f'minimum document frequency in target corpus ({min_document_frequency:,.0f})')
	if min_document_frequency_reference > 0:
		keyness_df = keyness_df.filter(pl.col('document_frequency_reference') >= min_document_frequency_reference)
		filtering_descriptors.append(f'minimum document frequency in reference corpus ({min_document_frequency_reference:,.0f})')
	
	if len(filtering_descriptors) > 0:
		formatted_data.append(f'Filtered tokens by {(", ".join(filtering_descriptors))}')

	unique_tokens = keyness_df.select(pl.len()).item()

	if statistical_significance_cut is not None and statistical_significance_cut > 0:
		p = statistical_significance_cut
		# bonferroni correction
		if apply_bonferroni:
			p_value_descriptor = f'Keywords filtered based on p-value {p} with Bonferroni correction (based on {unique_tokens} tests)'
			p = p / unique_tokens # adjust by criteria
		else:
			p_value_descriptor = f'Keywords filtered based on p-value: {p}'
		cut = chi2.ppf(1 - p, df=1)		
		keyness_df = keyness_df.filter(pl.col(statistical_significance_measure) > cut)
		formatted_data.append(p_value_descriptor)
		unique_tokens = keyness_df.select(pl.len()).item()

	if exclude_negative_keywords:
		keyness_df = keyness_df.filter(pl.col('relative_risk') >= 1)

	if order is None:
		order = statistical_significance_measure
	keyness_df = keyness_df.sort(order, descending=order_descending)
	if page_size == 0:
		rank_offset = 1
	else:
		keyness_df = keyness_df.slice((page_current-1)*page_size, page_size)
		rank_offset = (page_current-1) * page_size + 1

	if not show_document_frequency:
		keyness_df = keyness_df.drop('document_frequency', 'document_frequency_reference')
		columns.remove('document_frequency')
		columns.remove('document_frequency_reference')

	keyness_df = keyness_df.with_row_index(name='rank', offset=rank_offset)

	if normalize_by is not None:
		formatted_data.append(f'Normalized Frequency is per {normalize_by:,.0f} tokens')

	formatted_data.append(f'{total_descriptor} in target corpus: {target_count_tokens:,.0f}')
	formatted_data.append(f'{total_descriptor} in reference corpus: {reference_count_tokens:,.0f}')

	formatted_data.append(f'Keywords: {unique_tokens:,.0f}')
	if page_size != 0 and unique_tokens > page_size:
		formatted_data.append(f'Showing {page_size} rows')
		formatted_data.append(f'Page {page_current} of {unique_tokens // page_size + 1}')

	logger.info(f'Keywords report time: {(time.time() - start_time):.5f} seconds')

	if debug:
		columns = keyness_df.columns

	return Result(type = 'keywords', df=keyness_df.select(columns), title='Keywords', description=f'Target corpus: {self.corpus.name}, Reference corpus: {self.reference_corpus.name}', summary_data={}, formatted_data=formatted_data)


#### Note about restrict_tokens

When using restrict_tokens you may want to see full output of keyness results. To ensure you get output for all tokens, make sure that `statistical_significance_cut=None` and `exclude_negative_keywords=False`. 

Note: Polars 1.32.x seems to have introduced some instability about handling of is_in calls, which is required by restrict_tokens. Polars has been temporarily pinned to 1.31.0 to resolve this. 

#### Examples

See the note above about accessing this functionality through the [Conc](https://geoffford.nz/conc/api/conc.html) class.

In [ ]:
# load the target corpus
gardenparty = Corpus().load(path_to_gardenparty_corpus)

In [ ]:
# load the reference corpus
brown = Corpus().load(path_to_brown_corpus)

In [ ]:
# instantiate the Keyness class
keyness = Keyness(corpus = gardenparty, reference_corpus = brown)

In [ ]:
# generate and display the keywords report
keyness.keywords(show_document_frequency = True, statistical_significance_cut = 0.0001, apply_bonferroni = True, order_descending = True, page_current = 1).display()

Keywords 
 
 
 Target corpus: Garden Party Corpus, Reference corpus: Brown Corpus 
 
 
 Rank 
 Token 
 Frequency 
 Frequency Reference 
 Document Frequency 
 Document Frequency Reference 
 Normalized Frequency 
 Normalized Frequency Reference 
 Relative Risk 
 Log Ratio 
 Log Likelihood 
 
 
 
 
 1 
 she 
 1,171 
 2,060 
 15 
 216 
 196.76 
 21.02 
 9.36 
 3.23 
 2,710.68 
 
 
 2 
 her 
 937 
 2,887 
 15 
 253 
 157.44 
 29.45 
 5.35 
 2.42 
 1,442.33 
 
 
 3 
 josephine 
 117 
 0 
 1 
 0 
 19.66 
 0.00 
 3,853.78 
 11.91 
 669.34 
 
 
 4 
 said 
 514 
 1,944 
 15 
 315 
 86.37 
 19.83 
 4.35 
 2.12 
 648.89 
 
 
 5 
 n’t 
 522 
 2,016 
 15 
 286 
 87.71 
 20.57 
 4.26 
 2.09 
 644.51 
 
 
 6 
 you 
 642 
 3,265 
 15 
 293 
 107.87 
 33.31 
 3.24 
 1.70 
 566.70 
 
 
 7 
 it 
 1,021 
 6,991 
 15 
 498 
 171.56 
 71.33 
 2.41 
 1.27 
 552.39 
 
 
 8 
 oh 
 149 
 93 
 15 
 62 
 25.04 
 0.95 
 26.39 
 4.72 
 540.97 
 
 
 9 
 little 
 307 
 823 
 15 
 322 
 51.58 
 8.40 
 6.14 
 2.62 
 531.41 
 
 
 10 
 constantia 
 91 
 0 
 1 
 0 
 15.29 
 0.00 
 2,997.38 
 11.55 
 520.60 
 
 
 11 
 i 
 719 
 4,370 
 15 
 335 
 120.81 
 44.59 
 2.71 
 1.44 
 483.12 
 
 
 12 
 laura 
 86 
 14 
 2 
 6 
 14.45 
 0.14 
 101.17 
 6.66 
 412.65 
 
 
 13 
 isabel 
 71 
 1 
 2 
 1 
 11.93 
 0.01 
 1,169.31 
 10.19 
 395.76 
 
 
 14 
 grandma 
 73 
 15 
 2 
 5 
 12.27 
 0.15 
 80.15 
 6.32 
 339.03 
 
 
 15 
 was 
 1,102 
 9,931 
 15 
 466 
 185.17 
 101.32 
 1.83 
 0.87 
 307.65 
 
 
 16 
 fenella 
 49 
 0 
 1 
 0 
 8.23 
 0.00 
 1,613.98 
 10.66 
 280.32 
 
 
 17 
 dear 
 78 
 54 
 13 
 36 
 13.11 
 0.55 
 23.79 
 4.57 
 273.99 
 
 
 18 
 beryl 
 50 
 3 
 1 
 3 
 8.40 
 0.03 
 274.49 
 8.10 
 263.34 
 
 
 19 
 hammond 
 47 
 2 
 1 
 2 
 7.90 
 0.02 
 387.02 
 8.60 
 252.40 
 
 
 20 
 yes 
 87 
 109 
 14 
 74 
 14.62 
 1.11 
 13.15 
 3.72 
 241.33 
 
 
 
 
 
 Report based on word tokens 
 


 
 Frequent tokens with apostrophes have been normalized in reference corpus to match target usage 
 


 
 Keywords filtered based on p-value 0.0001 with Bonferroni correction (based on 5392 tests) 
 


 
 Normalized Frequency is per 10,000 tokens 
 


 
 Total word tokens in target corpus: 59,514 
 


 
 Total word tokens in reference corpus: 980,144 
 


 
 Keywords: 337 
 


 
 Showing 20 rows 
 


 
 Page 1 of 17

In [ ]:
#| hide
assert len(keyness.keywords(restrict_tokens = ['he', 'his', 'she', 'her', 'the'], order_descending = True, page_current = 1, exclude_negative_keywords = False).to_frame()) == 5

In [ ]:
#| hide
brown_listcorpus = ListCorpus().load(path_to_brown_listcorpus)
gardenparty_listcorpus = ListCorpus().load(path_to_gardenparty_listcorpus)
keyness_listcorpus = Keyness(corpus = gardenparty, reference_corpus = brown_listcorpus)
keyness_listcorpus = Keyness(corpus = gardenparty_listcorpus, reference_corpus = brown)
keyness_listcorpus = Keyness(corpus = gardenparty_listcorpus, reference_corpus = brown_listcorpus)
# min_document_frequency_reference = 5, 
keyness_listcorpus.keywords(show_document_frequency = True, statistical_significance_cut = 0.0001, apply_bonferroni = True, order_descending = True, page_current = 1).display()

Keywords 
 
 
 Target corpus: Garden Party Corpus, Reference corpus: Brown Corpus 
 
 
 Rank 
 Token 
 Frequency 
 Frequency Reference 
 Document Frequency 
 Document Frequency Reference 
 Normalized Frequency 
 Normalized Frequency Reference 
 Relative Risk 
 Log Ratio 
 Log Likelihood 
 
 
 
 
 1 
 she 
 1,171 
 2,060 
 15 
 216 
 196.76 
 21.02 
 9.36 
 3.23 
 2,710.68 
 
 
 2 
 her 
 937 
 2,887 
 15 
 253 
 157.44 
 29.45 
 5.35 
 2.42 
 1,442.33 
 
 
 3 
 josephine 
 117 
 0 
 1 
 0 
 19.66 
 0.00 
 3,853.78 
 11.91 
 669.34 
 
 
 4 
 said 
 514 
 1,944 
 15 
 315 
 86.37 
 19.83 
 4.35 
 2.12 
 648.89 
 
 
 5 
 n’t 
 522 
 2,016 
 15 
 286 
 87.71 
 20.57 
 4.26 
 2.09 
 644.51 
 
 
 6 
 you 
 642 
 3,265 
 15 
 293 
 107.87 
 33.31 
 3.24 
 1.70 
 566.70 
 
 
 7 
 it 
 1,021 
 6,991 
 15 
 498 
 171.56 
 71.33 
 2.41 
 1.27 
 552.39 
 
 
 8 
 oh 
 149 
 93 
 15 
 62 
 25.04 
 0.95 
 26.39 
 4.72 
 540.97 
 
 
 9 
 little 
 307 
 823 
 15 
 322 
 51.58 
 8.40 
 6.14 
 2.62 
 531.41 
 
 
 10 
 constantia 
 91 
 0 
 1 
 0 
 15.29 
 0.00 
 2,997.38 
 11.55 
 520.60 
 
 
 11 
 i 
 719 
 4,370 
 15 
 335 
 120.81 
 44.59 
 2.71 
 1.44 
 483.12 
 
 
 12 
 laura 
 86 
 14 
 2 
 6 
 14.45 
 0.14 
 101.17 
 6.66 
 412.65 
 
 
 13 
 isabel 
 71 
 1 
 2 
 1 
 11.93 
 0.01 
 1,169.31 
 10.19 
 395.76 
 
 
 14 
 grandma 
 73 
 15 
 2 
 5 
 12.27 
 0.15 
 80.15 
 6.32 
 339.03 
 
 
 15 
 was 
 1,102 
 9,931 
 15 
 466 
 185.17 
 101.32 
 1.83 
 0.87 
 307.65 
 
 
 16 
 fenella 
 49 
 0 
 1 
 0 
 8.23 
 0.00 
 1,613.98 
 10.66 
 280.32 
 
 
 17 
 dear 
 78 
 54 
 13 
 36 
 13.11 
 0.55 
 23.79 
 4.57 
 273.99 
 
 
 18 
 beryl 
 50 
 3 
 1 
 3 
 8.40 
 0.03 
 274.49 
 8.10 
 263.34 
 
 
 19 
 hammond 
 47 
 2 
 1 
 2 
 7.90 
 0.02 
 387.02 
 8.60 
 252.40 
 
 
 20 
 yes 
 87 
 109 
 14 
 74 
 14.62 
 1.11 
 13.15 
 3.72 
 241.33 
 
 
 
 
 
 Report based on word tokens 
 


 
 Frequent tokens with apostrophes have been normalized in reference corpus to match target usage 
 


 
 Keywords filtered based on p-value 0.0001 with Bonferroni correction (based on 5392 tests) 
 


 
 Normalized Frequency is per 10,000 tokens 
 


 
 Total word tokens in target corpus: 59,514 
 


 
 Total word tokens in reference corpus: 980,144 
 


 
 Keywords: 337 
 


 
 Showing 20 rows 
 


 
 Page 1 of 17

In [ ]:
#| hide
set_logger_state('quiet')
brown_listcorpus = ListCorpus().load(path_to_brown_listcorpus)
keyness_listcorpus = Keyness(corpus =  gardenparty, reference_corpus = brown_listcorpus)
keyness_listcorpus.keywords(show_document_frequency=True, min_document_frequency_reference = 7, handle_common_typographic_differences = True, page_size = 10, statistical_significance_cut = 0.0001, apply_bonferroni = True).display()
set_logger_state('quiet') #order = 'log_likelihood', order_descending = True, 

Keywords 
 
 
 Target corpus: Garden Party Corpus, Reference corpus: Brown Corpus 
 
 
 Rank 
 Token 
 Frequency 
 Frequency Reference 
 Document Frequency 
 Document Frequency Reference 
 Normalized Frequency 
 Normalized Frequency Reference 
 Relative Risk 
 Log Ratio 
 Log Likelihood 
 
 
 
 
 1 
 she 
 1,171 
 2,060 
 15 
 216 
 196.76 
 21.02 
 9.36 
 3.23 
 2,710.68 
 
 
 2 
 her 
 937 
 2,887 
 15 
 253 
 157.44 
 29.45 
 5.35 
 2.42 
 1,442.33 
 
 
 3 
 said 
 514 
 1,944 
 15 
 315 
 86.37 
 19.83 
 4.35 
 2.12 
 648.89 
 
 
 4 
 n’t 
 522 
 2,016 
 15 
 286 
 87.71 
 20.57 
 4.26 
 2.09 
 644.51 
 
 
 5 
 you 
 642 
 3,265 
 15 
 293 
 107.87 
 33.31 
 3.24 
 1.70 
 566.70 
 
 
 6 
 it 
 1,021 
 6,991 
 15 
 498 
 171.56 
 71.33 
 2.41 
 1.27 
 552.39 
 
 
 7 
 oh 
 149 
 93 
 15 
 62 
 25.04 
 0.95 
 26.39 
 4.72 
 540.97 
 
 
 8 
 little 
 307 
 823 
 15 
 322 
 51.58 
 8.40 
 6.14 
 2.62 
 531.41 
 
 
 9 
 i 
 719 
 4,370 
 15 
 335 
 120.81 
 44.59 
 2.71 
 1.44 
 483.12 
 
 
 10 
 was 
 1,102 
 9,931 
 15 
 466 
 185.17 
 101.32 
 1.83 
 0.87 
 307.65 
 
 
 
 
 
 Report based on word tokens 
 


 
 Frequent tokens with apostrophes have been normalized in reference corpus to match target usage 
 


 
 Filtered tokens by minimum document frequency in reference corpus (7) 
 


 
 Keywords filtered based on p-value 0.0001 with Bonferroni correction (based on 3008 tests) 
 


 
 Normalized Frequency is per 10,000 tokens 
 


 
 Total word tokens in target corpus: 59,514 
 


 
 Total word tokens in reference corpus: 980,144 
 


 
 Keywords: 250 
 


 
 Showing 10 rows 
 


 
 Page 1 of 26

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()